<img style="float: left;" src="resources/made.jpg" width="35%" height="35%">

# Академия MADE
## Семинар 8 часть 2.  Simple online and realtime tracking (SORT)
Иван Карпухин, ведущий программист-исследователь команды машинного зрения

<div style="clear:both;"></div>

В предыдущей части семинара мы настроили фильтр Калмана. В этой части реализуем простой алгоритм трекинга на его основе.

Для выполнения работы нужны следующие пакеты (Python 3):
* filterpy
* matplotlib
* numpy
* opencv-python
* tqdm
* pyyaml

Установить их можно командой:
```bash
pip3 install --user filterpy matplotlib numpy opencv-python tqdm pyyaml
```

In [1]:
# Раскомментируйте строчку ниже, чтобы установить зависимости.
#!pip3 install --user filterpy matplotlib numpy opencv-python tqdm pyyaml

In [2]:
from IPython.display import Video

import numpy as np
from matplotlib import pyplot as plt

import check
import seminar

# Исходное видео.
VIDEO_PATH = "data/sample.mp4"

# Видео с обнаруженными лицами.
DEMO_PATH = "data/sample-demo.mp4"

# Результаты работы детектора.
DETECTIONS_PATH = "data/sample-tracks.yaml"

# Видео, куда сохранится результат трекинга.
OUTPUT_PATH = "data/output-demo.mp4"

# Входные данные для трекера (copy-paste из части 1)

К нам поступил [видеофайл](https://www.youtube.com/watch?v=SvldnZ6qMGU) с записью людей на проходе в торговом центре. У нас имеется детектор лиц. Мы применили детектор лиц к видеофайлу и для каждого кадра получили набор описывающих прямоугольников (bounding box, BBox):

<img align="left" src="resources/video-frame.jpg" width="80%" height="80%">
<div style="clear:both;"></div>

На видео ниже полные результат работы детектора:

In [3]:
Video(DEMO_PATH, width=400)

Результаты работы детектора лиц хранятся в списке detections. Его длина соответствует числу кадров в видео (немного короче, т.к. в конце видео не нашлось детектов). Каждый элемент detections содержит список прямоуголников для кадра видео. Каждый прямоугольник задаётся четвёркой чисел: \[left, top, width, height\], или сокращённо \[l, t, w, h\].

In [4]:
FRAME_WIDTH, FRAME_HEIGHT, NUM_FRAMES, FRAME_RATE = seminar.video_probe(VIDEO_PATH)
print("Размер кадра: {}x{}".format(FRAME_WIDTH, FRAME_HEIGHT))
print("Число кадров:", NUM_FRAMES)
print("Частота кадров:", FRAME_RATE)

detections, _, markup = seminar.read_data(DETECTIONS_PATH)

print()
print("Общее число детектов:", sum(map(len, detections)))
print("Лица на 1-м кадре:", detections[0])
print("Лица на 60-м кадре:", detections[59])

Размер кадра: 960x720
Число кадров: 119
Частота кадров: 6.0

Общее число детектов: 393
Лица на 1-м кадре: [[228, 363, 61, 75]]
Лица на 60-м кадре: [[139, 601, 80, 109], [542, 134, 49, 58], [460, 99, 47, 63]]


# Simple online and realtime tracking (SORT)

Мы хотим сгруппировать прямоугольники с разных кадров для каждого человека.

Выполнять группировку мы будем последовательно: кадр за кадром. Связывать прямоугольники на соседних кадрах будем в два шага:

1. предскажем куда может переместиться прямоугольник за один кадр используя фильтр Калмана,
2. из обнаруженных на следующем кадре прямоугольников найдём ближайший к предсказанному.

На изображении ниже X и Y это координаты пикселя, а T - время. Разными цветами помечены прямоугольники разных лиц.

<img align="left" src="resources/matching.jpg" width="60%" height="60%">
<div style="clear:both;"></div>

Близость двух прямоугольников будем мерить используя Intersection Over Union (IoU):

<img align="left" src="resources/iou.jpg" width="35%" height="35%">
<div style="clear:both;"></div>

Используя IoU мы сможем померить близость между каждой парой предскзанных и наблюдаемых прямоугольников. Тогда мы сможем выбрать наиболее сильные связи решив задачу о максимальном паросочетании в двудольном графе:

<img align="left" src="resources/bipartite.jpg" width="70%" height="70%">
<div style="clear:both;"></div>

ЗАДАНИЕ. Реализуйте функцию, которая вычисляет Intersection Over Union (IoU) между всеми предсказаниями и обнаруженными прямоугольниками.

Поскольку время на задание ограничено, предлагаем использовать наиболее простые подходы. Быстродействие не учитывается.

In [5]:
def batch_iou(predictions, detections):
    """Вычислить Intersection over Union между каждым предсказанием и детектом.
    
    Все прямоугольники в формате [left, top, width, height].
    
    Вход:
    1. predictions: Предсказания фильтра Калмана, матрица размера (N, 4).
    2. detections: Обнаруженные детектором прямоугольники, матрица размера (K, 4).
    
    Выход: Матрица размера (N, K) с попарными IoU.
    """
    result = np.zeros((len(predictions), len(detections)))
    for i, bp in enumerate(predictions):
        for j, bd in enumerate(detections):
            inter_left = max(bp[0], bd[0])
            inter_top = max(bp[1], bd[1])
            inter_right = min(bp[0] + bp[2], bd[0] + bd[2])
            inter_bottom = min(bp[1] + bp[3], bd[1] + bd[3])
            inter_w = max(inter_right - inter_left, 0)
            inter_h = max(inter_bottom - inter_top, 0)
            inter_area = inter_w * inter_h
            union_area = bp[2] * bp[3] + bd[2] * bd[3] - inter_area
            result[i, j] = inter_area / union_area
    return result

assert check.check_batch_iou(batch_iou)

Результат: отлично!


Теперь приступим к поиску соответствий между предсказаниями и детектами.

В оригинальной реализации SORT используется Венгерский алгоритм. Предлагается реализовать более простой вариант: жадный алгоритм. Жадный алгоритм на каждом шаге добавляет соответствие с максимальным IoU, затем удаляет связанные прямоугольники из рассмотрения.

ЗАДАНИЕ. Реализовать жадный алгоритм поиска соответствий. Жадность алгоритма не проверяется. Любой разумный алгоритм должен пройти тесты.

Полезные функции:
```python

np.argmax(A, axis=None):
    """Возвращает индексы максимальных элементов в направлении оси axis."""
    
np.take_along_axis(A, indices, axis=None):
    """Позволяет выбрать элементы матрицы по результату np.argmax."""
```

In [6]:
def match_bboxes(iou):
    """Найти соответствия между предсказаниями и детектами.
    
    ВНИМАНИЕ: Связывать прямоугольники с нулевым IoU не нужно.
    
    В обозначениях ниже N - число предсказаний, K - число детектов.
    
    Вход: Матрица попарных значений IoU с размером (N, K).
    
    Выход: Список длины N, в котором для каждого предсказания указан номер соответствующего детекта.
           Если какое-то предсказание оказалось несвязанным, в списке нужно указать None.
    """
    iou = iou.copy()
    ninf = -1
    matches = [None] * len(iou)
    if iou.size == 0:
        return matches
    while True:
        candidates = np.argmax(iou, axis=1)
        values = np.take_along_axis(iou, candidates[:, None], 1)[:, 0]
        best_candidate = np.argmax(values)
        best_value = values[best_candidate]
        if best_value <= 0:
            break
        matches[best_candidate] = candidates[best_candidate]
        iou[best_candidate] = ninf
        iou[:, candidates[best_candidate]] = ninf
    return matches


assert check.check_match_bboxes(match_bboxes)

Результат: отлично!


Функция ниже релизует простой вариант алгоритма SORT.

1. Для каждого прямоугольника на первом кадре создается новый трек.

2. Для последующих кадров выполняется поиск соответствий с использоанием IoU.

3. Если какой-то трек не имеет соответствий более 2-х кадров, он заканчивается.

4. Если какой-то прямоугольник не попал в трек, для него создается новый трек.

In [7]:
from collections import defaultdict

from seminar import xysr2ltwh, ltwh2xysr, create_kalman_filter


def track_sort(detections):
    """Сгруппировать прямоугольники с разных кадров используя фильтр Калмана и IoU.
    
    Вход: Список ответов детектора для каждого кадра. Каждый ответ детектора это список
          прямоугольников в формате LTWH.
          
    Выход: Список меток прямоугольников для каждого кадра. Метки для каждого кадра это список
           целочисленных меток всех прямоугольников кадра.
    """
    num_tracks = 0
    track_frames = defaultdict(list)
    track_detection_ids = defaultdict(list)
    track_filters = {}
    for frame, frame_detections in enumerate(detections):
        print("Кадр", frame)
        
        # Закончить старые треки.
        for track in list(track_filters):
            last_track_frame = track_frames[track][-1]
            if last_track_frame < frame - 2:
                del track_filters[track]
                print("Трек {} завершён".format(track))
        
        # Предсказать следующие прямоугольники для треков.
        for filter in track_filters.values():
            filter.predict()
        active_tracks = list(track_filters)
        predictions = [xysr2ltwh(track_filters[i].x[:4]) for i in active_tracks]
        
        # Связать предсказания и детекты.
        iou = batch_iou(predictions, frame_detections)
        matches = match_bboxes(iou)
        print("Продолжено треков: {}".format(len([m for m in matches if m is not None])))
        
        # Обновить треки.
        for track_id, match in enumerate(matches):
            if match is None:
                continue
            track = active_tracks[track_id]
            track_frames[track].append(frame)
            track_detection_ids[track].append(match)
            track_filters[track].update(ltwh2xysr(frame_detections[match]))
            
        # Добавить новые треки.
        all_detections = set(range(len(frame_detections)))
        matched_detections = {m for m in matches if m is not None}
        unmatched_detections = all_detections - matched_detections
        for i in unmatched_detections:
            track_frames[num_tracks].append(frame)
            track_detection_ids[num_tracks].append(i)
            
            bbox = frame_detections[i]
            initial_state = list(ltwh2xysr(bbox)) + [0, 0, 0]
            track_filters[num_tracks] = create_kalman_filter(initial_state)
            
            print("Трек {} создан".format(num_tracks))
            num_tracks += 1
            
    # Сформировать ответ.
    labels = [[None] * len(frame_detections) for frame_detections in detections]
    for track in track_frames.keys():
        for frame, detection_id in zip(track_frames[track], track_detection_ids[track]):
            labels[frame][detection_id] = track
            
    # Проверим, что заполнили все метки.
    for frame_labels in labels:
        for label in frame_labels:
            assert label is not None

    return labels


labels = track_sort(detections)

Кадр 0
Продолжено треков: 0
Трек 0 создан
Кадр 1
Продолжено треков: 0
Трек 1 создан
Трек 2 создан
Кадр 2
Продолжено треков: 1
Трек 3 создан
Кадр 3
Трек 0 завершён
Продолжено треков: 2
Кадр 4
Трек 1 завершён
Продолжено треков: 1
Кадр 5
Продолжено треков: 1
Трек 4 создан
Кадр 6
Трек 3 завершён
Продолжено треков: 1
Кадр 7
Продолжено треков: 1
Трек 5 создан
Кадр 8
Трек 4 завершён
Продолжено треков: 2
Кадр 9
Продолжено треков: 2
Кадр 10
Продолжено треков: 2
Кадр 11
Продолжено треков: 2
Трек 6 создан
Кадр 12
Продолжено треков: 3
Кадр 13
Продолжено треков: 3
Кадр 14
Продолжено треков: 3
Кадр 15
Продолжено треков: 3
Кадр 16
Продолжено треков: 3
Трек 7 создан
Кадр 17
Продолжено треков: 2
Кадр 18
Продолжено треков: 2
Кадр 19
Трек 6 завершён
Трек 7 завершён
Продолжено треков: 2
Кадр 20
Продолжено треков: 2
Кадр 21
Продолжено треков: 2
Кадр 22
Продолжено треков: 2
Кадр 23
Продолжено треков: 2
Кадр 24
Продолжено треков: 2
Кадр 25
Продолжено треков: 2
Трек 8 создан
Кадр 26
Продолжено треков: 2
Кадр 

In [8]:
error = seminar.eval_mismatch_rate(labels, markup)
print("Доля несоответствий с разметкой: {:.2f}".format(error))

Доля несоответствий с разметкой: 0.17


# Визуализация

In [9]:
seminar.render_video_bboxes(VIDEO_PATH, OUTPUT_PATH, detections, labels)

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 118/119 [00:01<00:00, 97.52it/s]


На видео прямоугольники разных треков помечены разным цветом.

Мигание цвета на каких-то лицах - это ошибки трекинга. Более точная настройка параметров (STD ошибок в фильтре Калмана) может решить проблему. Часть проблем нельзя решить методом SORT на видео с низким FPS.

In [10]:
print(OUTPUT_PATH)
Video(OUTPUT_PATH)

data/output-demo.mp4


Если видео не отображается, откройте его самостоятельно через плеер.

ВОПРОС. В каких случаях трекинг работает хорошо, а в каких плохо?

# Ссылки

## Фильтр Калмана

https://habr.com/ru/post/166693/

https://en.wikipedia.org/wiki/Kalman_filter

## Статьи

SORT: https://arxiv.org/abs/1602.00763

DeepSORT: https://arxiv.org/abs/1703.07402

## Реализации

Kalman: https://filterpy.readthedocs.io/en/latest/kalman/KalmanFilter.html

SORT: https://github.com/abewley/sort

DeepSORT: https://github.com/nwojke/deep_sort